In [1]:
import torch

In [2]:
def IPOT_torch(C, n, m, miu, nu, beta=0.5):
    
    # C is the distance matrix
    # c: n by m
    # miu: bs * n
    
    sigma = torch.ones(int(m), 1).float().cuda()/m # bs * m * 1
    T = torch.ones(n, m).cuda()    # Transport plan
    C = torch.exp(-C/beta).float() # Transport cost 
    for t in range(20):
        T = C * T # n * m
        for k in range(1):
            delta = miu / torch.squeeze(torch.matmul(T, sigma))
            sigma = torch.unsqueeze(nu,1) / torch.matmul(torch.transpose(T,0,1), torch.unsqueeze(delta,1))
        T = torch.unsqueeze(delta,1) * T * sigma.transpose(1,0)
    return T.detach()

In [3]:
def IPOT_distance_torch(C, n, m, miu, nu):
    T = IPOT_torch(C, n, m, miu, nu)
    distance = torch.trace(torch.mm(torch.transpose(C,0,1), T))
	return -distance

TabError: inconsistent use of tabs and spaces in indentation (<ipython-input-3-5ac7e4e170c6>, line 4)

# Implementation

In [4]:
from ot import *
import torch.nn as nn

In [94]:
# source x target y
x = torch.randn(128,36,1024)
y = torch.randn(128,14,1024)

x = torch.randn(128,512,1)
y = torch.randn(128,768,1)
# x = torch.randn(128,1, 512)
# y = torch.randn(128,1, 768)

In [95]:
# self.v_net = FCNet([v_dim, h_dim * self.k], act=act, dropout=dropout[0])
# self.q_net = FCNet([q_dim, h_dim * self.k], act=act, dropout=dropout[0])

In [96]:
cos_distance = cost_matrix_batch_torch(x.transpose(2, 1), y.transpose(2, 1))
print("after: ", cos_distance.shape)
cos_distance = cos_distance.transpose(1,2)
print("before: ", cos_distance.shape)

after:  torch.Size([128, 768, 512])
before:  torch.Size([128, 512, 768])


In [97]:
h_out = 32
h_dim = 32
k = 3 # ?
h_mat = nn.Parameter(torch.Tensor(1, h_out, 1, h_dim * k).normal_())
h_bias = nn.Parameter(torch.Tensor(1, h_out, 1, 1).normal_())
# h_net = weight_norm(nn.Linear(h_dim * self.k, h_out), dim=None)
print(h_mat.shape, h_bias.shape)

torch.Size([1, 32, 1, 96]) torch.Size([1, 32, 1, 1])


In [98]:
logits = torch.einsum('xhyk,bvk,bqk->bhvq', (h_mat, x, y)) + h_bias
print(logits.shape)

torch.Size([128, 32, 512, 768])


In [99]:
beta = 0.1
min_score = cos_distance.min()
max_score = cos_distance.max()
threshold = min_score + beta * (max_score - min_score)
cos_dist = torch.nn.functional.relu(cos_distance - threshold)

In [100]:
print(x.size(0), x.size(1), y.size(1))

128 512 768


In [101]:
wd = - IPOT_distance_torch_batch_uniform(C=cos_dist, bs=x.size(0), n=x.size(1), m=y.size(1), iteration=30)
print(wd.shape)

torch.Size([128, 1])


In [102]:
gwd = GW_distance_uniform(x.transpose(2,1), y.transpose(2,1))

In [103]:
twd = .1 * torch.mean(gwd) + .1 * torch.mean(wd)

### Wasserstein Distance

In [104]:
C=cos_dist
bs=x.size(0)
n=x.size(1)
m=y.size(1)
iteration=30

In [105]:
print("sigma: ", sigma.shape)
print("delta: ", delta.shape)

sigma:  torch.Size([128, 768, 1])
delta:  torch.Size([128, 512, 1])


In [106]:
# What is k?
for t in range(iteration):
    Q = A * T # bs * n * m
    for k in range(1):
        delta = 1 / (n * torch.bmm(Q, sigma))
        a = torch.bmm(torch.transpose(Q, 1, 2), delta)
        sigma = 1/ m * a
    T = delta * Q * sigma.transpose(2, 1)


### Gromove-Wasserstein Distance

In [107]:
m = x.size(1)
n = y.size(1)
bs = x.size(0)
p = (torch.ones(bs, m, 1)/m)
q = (torch.ones(bs, n, 1)/n)

print(m, n, bs, p.shape, q.shape)

512 768 128 torch.Size([128, 512, 1]) torch.Size([128, 768, 1])


In [108]:
Cs = cos_batch_torch(x.transpose(2,1), x.transpose(2,1))
Ct = cos_batch_torch(y.transpose(2,1), y.transpose(2,1))
# pdb.set_trace()
bs = Cs.size(0)
m = Ct.size(2)
n = Cs.size(2)
print(Cs.shape, Ct.shape, bs, m, n)

torch.Size([128, 512, 512]) torch.Size([128, 768, 768]) 128 768 512


In [109]:
one_m = torch.ones(bs, m, 1)
one_n = torch.ones(bs, n, 1)

In [112]:
Cs.shape

torch.Size([128, 512, 512])

In [111]:
torch.bmm(Cs**2, p).shape

torch.Size([128, 512, 1])

In [114]:
torch.bmm(torch.bmm(Cs**2, p), torch.transpose(one_m, 1, 2)).shape

torch.Size([128, 512, 768])

In [118]:
Cst = torch.bmm(torch.bmm(Cs**2, p), torch.transpose(one_m, 1, 2)) + \
        torch.bmm(one_n, torch.bmm(torch.transpose(q,1,2), torch.transpose(Ct**2, 1, 2))) # bs by n by m

In [121]:
gamma = torch.bmm(p, q.transpose(2,1)) # outer product, init

In [123]:
gamma.shape

torch.Size([128, 512, 768])

In [124]:
for i in range(iteration):
    C_gamma = Cst - 2 * torch.bmm(torch.bmm(Cs, gamma), torch.transpose(Ct, 1, 2))

In [130]:
one_m = torch.ones(bs, m, 1)
one_n = torch.ones(bs, n, 1)

Cst = torch.bmm(torch.bmm(Cs**2, p), torch.transpose(one_m, 1, 2)) + \
      torch.bmm(one_n, torch.bmm(torch.transpose(q,1,2), torch.transpose(Ct**2, 1, 2))) # bs by n by m
gamma = torch.bmm(p, q.transpose(2,1)) # outer product, init
for i in range(iteration):
    C_gamma = Cst - 2 * torch.bmm(torch.bmm(Cs, gamma), torch.transpose(Ct, 1, 2))
    gamma = IPOT_torch_batch_uniform(C_gamma, bs, n, m, beta=beta, iteration=20)
Cgamma = Cst - 2 * torch.bmm(torch.bmm(Cs, gamma), torch.transpose(Ct, 1, 2))

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat2 in method wrapper_bmm)